Instalando Pacote

In [ ]:
!git clone https://github.com/huggingface/transformers;
!cd transformers; pip3 install .

Cloning into 'transformers'...
remote: Enumerating objects: 94730, done.
remote: Total 94730 (delta 0), reused 0 (delta 0), pack-reused 94730
Receiving objects: 100% (94730/94730), 85.28 MiB | 11.57 MiB/s, done.
Resolving deltas: 100% (69509/69509), done.
Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 77 kB 3.1 MB/s 
     |████████████████████████████████| 895 kB 8.5 MB/s 
     |████████████████████████████████| 596 kB 54.6 MB/s 
     |████████████

In [ ]:
! pip install simpletransformers

     |████████████████████████████████| 249 kB 4.1 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 325 kB 56.6 MB/s 
     |████████████████████████████████| 1.8 MB 51.6 MB/s 
     |████████████████████████████████| 1.2 MB 53.6 MB/s 
     |████████████████████████████████| 10.1 MB 58.6 MB/s 
     |████████████████████████████████| 181 kB 76.6 MB/s 
     |████████████████████████████████| 144 kB 66.6 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 136 kB 71.6 MB/s 
     |████████████████████████████████| 1.1 MB 53.2 MB/s 
     |████████████████████████████████| 212 kB 34.0 MB/s 
     |████████████████████████████████| 127 kB 54.6 MB/s 
     |████████████████████████████████| 271 kB 63.4 MB/s 
     |████████████████████████████████| 144 kB 27.6 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 111 kB 68.6 MB/s 
     |██████████████

In [ ]:
! pip install datasets

Puxando Dataset de Discurso de Ódio

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
import pandas as pd
import transformers
from transformers import AutoModel, BertTokenizerFast
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer



# link: https://huggingface.co/datasets/ag_news

In [ ]:
from datasets import load_dataset

dataset = load_dataset("emotion")

Using custom data configuration default


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
X_train = dataset['validation']['text']
X_test = dataset['test']['text']
y_train = dataset['validation']['label']
y_test = dataset['test']['label']

In [ ]:
# Quatro classes

# 0:"World"
# 1:"Sports"
# 2:"Business"
# 3:"Sci/Tech"


In [ ]:
def puxa_dados_normal(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()
  
  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()
  

  parafrase = csv[['parafraseador', 'label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase.rename(columns={"parafraseador": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values
  

  return X, y 


In [8]:
def desempenho_LSTM(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  mask_zero= True,
                                  output_dim=embedding_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))
  
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  
  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  
  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [9]:
def desempenho_CNN(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  mask_zero= True,
                                  output_dim=embedding_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Conv1D(filters=100, kernel_size = 3, activation = 'relu', 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu', 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))
  
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  
  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  
  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [10]:
# Evaluate the model
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
      return f1_score(labels, preds, average='macro')


In [11]:
def desempenho_transformers(X_train, X_test, y_train, y_test,classes, model,type_model, epochs, batch, max_len):
  from simpletransformers.classification import ClassificationModel
  import pandas as pd
  import logging
  import sklearn

  train_df = pd.DataFrame(X_train)
  train_df['target'] = y_train

  eval_df = pd.DataFrame(X_test)
  eval_df['target'] = y_test



  # Create a ClassificationModel
  model = ClassificationModel(type_model, model, num_labels=classes, args={'num_train_epochs':epochs,
                                                                           'train_batch_size':batch,
                                                                           'max_seq_length':max_len,
                                                                          #  "fp16":False,
                                                                           'overwrite_output_dir': True}) 

  # Train the model
  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score,f1=f1_multiclass)

  return result['acc'], result['f1']





In [12]:
def desempenho_SVM(X_train, X_test, y_train, y_text):
  text_clf_svm = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf-svm', SVC(gamma='auto')),])
  _ = text_clf_svm.fit(X_train, y_train)
  predicted_svm = text_clf_svm.predict(X_test)
  acc = accuracy_score(y_test, predicted_svm)
  f1 = f1_score(y_test, predicted_svm, average='macro')

  return acc, f1


In [13]:

# X_test = dataset['test']['text']
# y_test = dataset['test']['label-coarse']
y_test = np.array(y_test)


In [14]:
y_test.shape

(2000,)

In [15]:
def puxa_original(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')

  X = csv['texto'].values
  y = csv['label'].values
  

  return X, y 

In [16]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_keyarg'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_bert'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_deep_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_parrot'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_protaug'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  if j == 0:
    X_train,  y_train = puxa_original(arqs[j])
  else:
    X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_emotions_100_original.csv


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

  0%|          | 0/11997 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/375 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
data = {'Aug': [ 'sem', 'key', 'eda', 'emb','bert','back_trans', 'deep_back_trans', 'parrot', 'protaug'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,BERT_ACC,BERT_F1
0,sem,0.5950,0.244382
1,key,0.8690,0.787649
2,eda,0.8885,0.824023
3,emb,0.8835,0.831177
4,bert,0.8730,0.821655
5,back_trans,0.8190,0.723345
6,deep_back_trans,0.8160,0.683191
7,parrot,0.8570,0.786930
8,protaug,0.8825,0.820147


In [ ]:
repete = 1
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_resultados'+str(repete)+'.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

In [ ]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_keyarg'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_bert'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_deep_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_parrot'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_protaug'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  if j == 0:
    X_train,  y_train = puxa_original(arqs[j])
  else:
    X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

data = {'Aug': [ 'sem', 'key', 'eda', 'emb','bert','back_trans', 'deep_back_trans', 'parrot', 'protaug'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)


repete = 2
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_resultados'+str(repete)+'.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_emotions_100_original.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/321 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

BERT
ACC:0.8925
F1:0.8411526859425109


In [ ]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_keyarg'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_bert'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_deep_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_parrot'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_protaug'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  if j == 0:
    X_train,  y_train = puxa_original(arqs[j])
  else:
    X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

data = {'Aug': [ 'sem', 'key', 'eda', 'emb','bert','back_trans', 'deep_back_trans', 'parrot', 'protaug'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)


repete = 3
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_resultados'+str(repete)+'.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_emotions_100_original.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/321 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

BERT
ACC:0.8865
F1:0.8340257661046139


In [ ]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_keyarg'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_bert'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_deep_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_parrot'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_protaug'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  if j == 0:
    X_train,  y_train = puxa_original(arqs[j])
  else:
    X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

data = {'Aug': [ 'sem', 'key', 'eda', 'emb','bert','back_trans', 'deep_back_trans', 'parrot', 'protaug'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)


repete = 4
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_resultados'+str(repete)+'.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_emotions_100_original.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/321 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

BERT
ACC:0.897
F1:0.8468813932318949


In [ ]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_keyarg'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_bert'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_deep_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_parrot'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_protaug'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  if j == 0:
    X_train,  y_train = puxa_original(arqs[j])
  else:
    X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

data = {'Aug': [ 'sem', 'key', 'eda', 'emb','bert','back_trans', 'deep_back_trans', 'parrot', 'protaug'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)


repete = 5
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_resultados'+str(repete)+'.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_emotions_100_original.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/321 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

BERT
ACC:0.895
F1:0.8437547906287253


In [ ]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_keyarg'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_bert'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_deep_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_parrot'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_protaug'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  if j == 0:
    X_train,  y_train = puxa_original(arqs[j])
  else:
    X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

data = {'Aug': [ 'sem', 'key', 'eda', 'emb','bert','back_trans', 'deep_back_trans', 'parrot', 'protaug'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)


repete = 6
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_resultados'+str(repete)+'.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_emotions_100_original.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/321 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

BERT
ACC:0.895
F1:0.8442148445330663


In [ ]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_keyarg'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_bert'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_deep_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_parrot'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_protaug'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  if j == 0:
    X_train,  y_train = puxa_original(arqs[j])
  else:
    X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

data = {'Aug': [ 'sem', 'key', 'eda', 'emb','bert','back_trans', 'deep_back_trans', 'parrot', 'protaug'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)


repete = 7
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_resultados'+str(repete)+'.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_emotions_100_original.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/321 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

BERT
ACC:0.892
F1:0.833285802970802


In [ ]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_keyarg'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_bert'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_deep_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_parrot'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_protaug'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  if j == 0:
    X_train,  y_train = puxa_original(arqs[j])
  else:
    X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

data = {'Aug': [ 'sem', 'key', 'eda', 'emb','bert','back_trans', 'deep_back_trans', 'parrot', 'protaug'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)


repete = 8
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_resultados'+str(repete)+'.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_emotions_100_original.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/12000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/375 [00:00<?, ?it/s]

In [ ]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_keyarg'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_bert'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_deep_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_parrot'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_protaug'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  if j == 0:
    X_train,  y_train = puxa_original(arqs[j])
  else:
    X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

data = {'Aug': [ 'sem', 'key', 'eda', 'emb','bert','back_trans', 'deep_back_trans', 'parrot', 'protaug'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)


repete = 9
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_resultados'+str(repete)+'.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_emotions_100_original.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/321 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

BERT
ACC:0.889
F1:0.8385619156119182


In [ ]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_original.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_keyarg'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_emb'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_bert'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_deep_back_trans'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_parrot'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_protaug'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  if j == 0:
    X_train,  y_train = puxa_original(arqs[j])
  else:
    X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

data = {'Aug': [ 'sem', 'key', 'eda', 'emb','bert','back_trans', 'deep_back_trans', 'parrot', 'protaug'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)


repete = 10
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_emotions_'+str(perc)+'_resultados'+str(repete)+'.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_emotions_100_original.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/321 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

BERT
ACC:0.891
F1:0.8345025592503967
